<a href="https://colab.research.google.com/github/sh-0620/dacon-recommender-system/blob/main/recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings

from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_path = '/content/drive/MyDrive/데이콘/기사추천시스템/open/'

In [4]:
article_info = pd.read_csv(data_path + 'article_info.csv')

In [5]:
article_info #기사 정보(userid = 기사 작성자)

,articleID,Title,Content,Format,Language,userID,userCountry,userRegion
0,ARTICLE_0000,19 Tips For Everyday Git Use,I've been using git full time for the past 4 y...,HTML,en,USER_0683,NaN,NaN
1,ARTICLE_0001,Intel buys computer vision startup Itseez to i...,Intel has acquired computer vision and machine...,HTML,en,USER_1129,NaN,NaN
2,ARTICLE_0002,Practical End-to-End Testing with Protractor,One of the reasons AngularJS is so great to wo...,HTML,en,USER_0256,NaN,NaN
3,ARTICLE_0003,Corporate venture growth in Brazil is another ...,Despite recent positive news and a renewed int...,HTML,en,USER_1304,NaN,NaN
4,ARTICLE_0004,Cross-channel user experiences with Drupal (aw...,"Last year around this time, I wrote that The B...",HTML,en,USER_0336,NaN,NaN
...,...,...,...,...,...,...,...,...
3003,ARTICLE_3003,Como consumir conteúdo de qualidade em iOS - C...,"Quando iniciei minha jornada em Swift, saindo ...",HTML,pt,USER_0882,BR,MG
3004,ARTICLE_3004,Aurelia 1.0 is Here!!!,It's been an amazing journey to get here and i...,HTML,en,USER_0220,NaN,NaN
3005,ARTICLE_3005,Lessons from converting an app to 100% Kotlin ...,This is part one in a series of posts about Ko...,HTML,en,USER_1010,BR,SP
3006,ARTICLE_3006,ITA está oferecendo 10 cursos gratuitos a dist...,"O Instituto Tecnológico de Aeronáutica (ITA) ,...",HTML,pt,USER_1210,NaN,NaN


In [6]:
article_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008 entries, 0 to 3007
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   articleID    3008 non-null   object
 1   Title        3008 non-null   object
 2   Content      3008 non-null   object
 3   Format       3008 non-null   object
 4   Language     3008 non-null   object
 5   userID       3008 non-null   object
 6   userCountry  659 non-null    object
 7   userRegion   657 non-null    object
dtypes: object(8)
memory usage: 188.1+ KB


In [7]:
article_info

,articleID,Title,Content,Format,Language,userID,userCountry,userRegion
0,ARTICLE_0000,19 Tips For Everyday Git Use,I've been using git full time for the past 4 y...,HTML,en,USER_0683,NaN,NaN
1,ARTICLE_0001,Intel buys computer vision startup Itseez to i...,Intel has acquired computer vision and machine...,HTML,en,USER_1129,NaN,NaN
2,ARTICLE_0002,Practical End-to-End Testing with Protractor,One of the reasons AngularJS is so great to wo...,HTML,en,USER_0256,NaN,NaN
3,ARTICLE_0003,Corporate venture growth in Brazil is another ...,Despite recent positive news and a renewed int...,HTML,en,USER_1304,NaN,NaN
4,ARTICLE_0004,Cross-channel user experiences with Drupal (aw...,"Last year around this time, I wrote that The B...",HTML,en,USER_0336,NaN,NaN
...,...,...,...,...,...,...,...,...
3003,ARTICLE_3003,Como consumir conteúdo de qualidade em iOS - C...,"Quando iniciei minha jornada em Swift, saindo ...",HTML,pt,USER_0882,BR,MG
3004,ARTICLE_3004,Aurelia 1.0 is Here!!!,It's been an amazing journey to get here and i...,HTML,en,USER_0220,NaN,NaN
3005,ARTICLE_3005,Lessons from converting an app to 100% Kotlin ...,This is part one in a series of posts about Ko...,HTML,en,USER_1010,BR,SP
3006,ARTICLE_3006,ITA está oferecendo 10 cursos gratuitos a dist...,"O Instituto Tecnológico de Aeronáutica (ITA) ,...",HTML,pt,USER_1210,NaN,NaN


In [8]:
view_log = pd.read_csv(data_path + 'view_log.csv')

In [9]:
view_log #사용자-기사 데이터

,userID,articleID,userRegion,userCountry
0,USER_0000,ARTICLE_0661,NY,US
1,USER_0000,ARTICLE_2316,NY,US
2,USER_0000,ARTICLE_1345,NY,US
3,USER_0000,ARTICLE_1089,NY,US
4,USER_0000,ARTICLE_1484,NY,US
...,...,...,...,...
42712,USER_1420,ARTICLE_0682,SP,BR
42713,USER_1420,ARTICLE_2179,SP,BR
42714,USER_1420,ARTICLE_1848,SP,BR
42715,USER_1420,ARTICLE_0030,SP,BR


In [10]:
view_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42717 entries, 0 to 42716
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   userID       42717 non-null  object
 1   articleID    42717 non-null  object
 2   userRegion   42717 non-null  object
 3   userCountry  42717 non-null  object
dtypes: object(4)
memory usage: 1.3+ MB


# TF-IDF

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

In [12]:
# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000) #stop_words='english'는 영어 불용어를 제거하고, max_features=5000는 최대 5000개의 단어 특성을 사용한다는 의미입니다.
tfidf_matrix = tfidf_vectorizer.fit_transform(article_info['Title'])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [ ]:
def get_personalized_recommendations(user_id, cosine_sim=cosine_sim, articles_df=article_info):
    # 사용자가 읽은 기사 목록 가져오기
    user_articles = view_log[view_log['userID'] == user_id]['articleID'].tolist()

    # 사용자가 읽은 기사의 인덱스 가져오기
    article_indices = article_info[article_info['articleID'].isin(user_articles)].index.tolist()

    # 유사도 점수 합산
    sim_scores = cosine_sim[article_indices].sum(axis=0)

    # 유사도 점수를 기준으로 기사 정렬
    ranked_indices = sim_scores.argsort()[::-1]

    # 상위 5개 기사 추천
    recommended_article_indices = ranked_indices[:5]

    return articles_df.iloc[recommended_article_indices]



In [ ]:
get_personalized_recommendations('USER_1420')['articleID'].values

array(['ARTICLE_1351', 'ARTICLE_1891', 'ARTICLE_1788', 'ARTICLE_0445',
       'ARTICLE_2409'], dtype=object)

In [ ]:
recommendations = []
for idx, user in enumerate(view_log.userID.unique()):
    recommendations.append([user, get_personalized_recommendations(user).articleID.values])

In [ ]:
recommendations = pd.DataFrame(recommendations, columns=['userID', 'articleID'])

In [ ]:
recommendations = recommendations.explode('articleID').reset_index(drop=True)


In [ ]:
submission = recommendations.to_csv(data_path + 'submission_v2.csv')

In [ ]:
submission

In [ ]:
data_path

'/content/drive/MyDrive/데이콘/기사추천시스템/open/'

# word2vec

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

## 1) no remove stopword

In [16]:
# 기사 제목을 단어 리스트로 변환
sentences = [title.split() for title in article_info['Title']]

# Word2Vec 모델 학습
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# 각 문서의 벡터를 평균하여 문서 벡터 생성
def document_vector(doc):
    # 해당 문서의 단어 벡터를 가져와서 평균을 계산
    return np.mean([w2v_model.wv[word] for word in doc if word in w2v_model.wv], axis=0)

# 모든 기사의 벡터를 계산
article_vectors = np.array([document_vector(title.split()) for title in article_info['Title']])


In [18]:
# 코사인 유사도 계산
cosine_sim = cosine_similarity(article_vectors)

def get_personalized_recommendations_w2v(user_id, cosine_sim=cosine_sim, articles_df=article_info):
    # 사용자가 읽은 기사 목록 가져오기
    user_articles = view_log[view_log['userID'] == user_id]['articleID'].tolist()

    # 사용자가 읽은 기사의 인덱스 가져오기
    article_indices = article_info[article_info['articleID'].isin(user_articles)].index.tolist()

    # 유사도 점수 합산
    sim_scores = cosine_sim[article_indices].sum(axis=0)

    # 유사도 점수를 기준으로 기사 정렬
    ranked_indices = sim_scores.argsort()[::-1]

    # 상위 5개 기사 추천
    recommended_article_indices = ranked_indices[:5]

    return articles_df.iloc[recommended_article_indices]

# 예시 사용법
user_id = 'USER_0000'
recommended_articles = get_personalized_recommendations_w2v(user_id)
print(recommended_articles)


         articleID                                              Title  \
826   ARTICLE_0826  How to forecast demand with Google BigQuery, p...   
1861  ARTICLE_1861  How Technology Hijacks People's Minds - from a...   
150   ARTICLE_0150  How a Japanese cucumber farmer is using deep l...   
1238  ARTICLE_1238  Google Cloud Dataproc - the fast, easy and saf...   
844   ARTICLE_0844  Brazilian Government and Ericsson to innovate ...   

                                                Content Format Language  \
826   Demand forecasting is something that every bus...   HTML       en   
1861  How Technology Hijacks People's Minds - from a...   HTML       en   
150   It's not hyperbole to say that use cases for m...   HTML       en   
1238  Posted by James Malone, Product Manager Please...   HTML       en   
844   Public-private partnership supports creation o...   HTML       en   

         userID userCountry userRegion  
826   USER_0222         NaN        NaN  
1861  USER_0416         NaN 

In [19]:
recommendations_word2vec = []
for idx, user in enumerate(view_log.userID.unique()):
    recommendations_word2vec.append([user, get_personalized_recommendations_w2v(user).articleID.values])

In [20]:
recommendations_word2vec = pd.DataFrame(recommendations_word2vec, columns=['userID', 'articleID'])

In [21]:
recommendations_word2vec

,userID,articleID
0,USER_0000,"[ARTICLE_0826, ARTICLE_1861, ARTICLE_0150, ART..."
1,USER_0001,"[ARTICLE_1437, ARTICLE_1861, ARTICLE_0906, ART..."
2,USER_0002,"[ARTICLE_2990, ARTICLE_2204, ARTICLE_0906, ART..."
3,USER_0003,"[ARTICLE_2994, ARTICLE_0782, ARTICLE_1104, ART..."
4,USER_0004,"[ARTICLE_1861, ARTICLE_2198, ARTICLE_0763, ART..."
...,...,...
1410,USER_1416,"[ARTICLE_0399, ARTICLE_2703, ARTICLE_0813, ART..."
1411,USER_1417,"[ARTICLE_1486, ARTICLE_2198, ARTICLE_2868, ART..."
1412,USER_1418,"[ARTICLE_0535, ARTICLE_1997, ARTICLE_0774, ART..."
1413,USER_1419,"[ARTICLE_0439, ARTICLE_0867, ARTICLE_0693, ART..."


In [22]:
recommendations_word2vec2 = recommendations_word2vec.explode('articleID').reset_index(drop=True)


In [ ]:
recommendations_word2vec2.to_csv(data_path + 'submission_w2v_v2.csv')

In [23]:
recommendations_word2vec2

,userID,articleID
0,USER_0000,ARTICLE_0826
1,USER_0000,ARTICLE_1861
2,USER_0000,ARTICLE_0150
3,USER_0000,ARTICLE_1238
4,USER_0000,ARTICLE_0844
...,...,...
7070,USER_1420,ARTICLE_1437
7071,USER_1420,ARTICLE_1486
7072,USER_1420,ARTICLE_2033
7073,USER_1420,ARTICLE_1913


## 2) remove stopword

In [29]:
# nltk 불용어 다운로드
import re
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# 기사 제목을 전처리하고 단어 리스트로 변환
def preprocess(text):
    # 소문자 변환, 특수문자 제거, 불용어 제거
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return words

sentences = [preprocess(title) for title in article_info['Title']]

# Word2Vec 모델 학습
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# 각 문서의 벡터를 평균하여 문서 벡터 생성
def document_vector(doc):
    # 해당 문서의 단어 벡터를 가져와서 평균을 계산
    return np.mean([w2v_model.wv[word] for word in doc if word in w2v_model.wv], axis=0)

# 모든 기사의 벡터를 계산
article_vectors = np.array([document_vector(preprocess(title)) for title in article_info['Title']])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
# 코사인 유사도 계산
cosine_sim = cosine_similarity(article_vectors)

def get_personalized_recommendations(user_id, cosine_sim=cosine_sim, articles_df=article_info):
    # 사용자가 읽은 기사 목록 가져오기
    user_articles = view_log[view_log['userID'] == user_id]['articleID'].tolist()

    # 사용자가 읽은 기사의 인덱스 가져오기
    article_indices = article_info[article_info['articleID'].isin(user_articles)].index.tolist()

    # 유사도 점수 합산
    sim_scores = cosine_sim[article_indices].sum(axis=0)

    # 유사도 점수를 기준으로 기사 정렬
    ranked_indices = sim_scores.argsort()[::-1]

    # 사용자가 이미 읽은 기사 제외
    recommended_article_indices = [idx for idx in ranked_indices if idx not in article_indices][:5]

    return articles_df.iloc[recommended_article_indices]

# 예시 사용법
user_id = 'USER_0000'
recommended_articles = get_personalized_recommendations(user_id)
print(recommended_articles)


         articleID                                              Title  \
2572  ARTICLE_2572  Google Save, serviço para salvar coisas da web...   
150   ARTICLE_0150  How a Japanese cucumber farmer is using deep l...   
2461  ARTICLE_2461                   The Blockchain is the new Google   
1360  ARTICLE_1360  Google Cloud Machine Learning family grows wit...   
252   ARTICLE_0252  Google lança teclado virtual para iPhone com r...   

                                                Content Format Language  \
2572  Restrito aos Estados Unidos desde seu lançamen...   HTML       pt   
150   It's not hyperbole to say that use cases for m...   HTML       en   
2461  The following is an excerpt from The Business ...   HTML       en   
1360  Google Cloud Machine Learning is one of our fa...   HTML       en   
252   O Google anunciou hoje (12) o lançamento do Gb...   HTML       pt   

         userID userCountry userRegion  
2572  USER_0017         NaN        NaN  
150   USER_0222         NaN 

In [32]:
recommendations_word2vec = []
for idx, user in enumerate(view_log.userID.unique()):
    recommendations_word2vec.append([user, get_personalized_recommendations(user).articleID.values])

In [33]:
recommendations_word2vec = pd.DataFrame(recommendations_word2vec, columns=['userID', 'articleID'])

In [34]:
recommendations_word2vec

,userID,articleID
0,USER_0000,"[ARTICLE_2572, ARTICLE_0150, ARTICLE_2461, ART..."
1,USER_0001,"[ARTICLE_1104, ARTICLE_2461, ARTICLE_2672, ART..."
2,USER_0002,"[ARTICLE_2672, ARTICLE_0906, ARTICLE_0767, ART..."
3,USER_0003,"[ARTICLE_2672, ARTICLE_0012, ARTICLE_0767, ART..."
4,USER_0004,"[ARTICLE_2672, ARTICLE_0012, ARTICLE_0767, ART..."
...,...,...
1410,USER_1416,"[ARTICLE_0817, ARTICLE_0389, ARTICLE_0843, ART..."
1411,USER_1417,"[ARTICLE_2672, ARTICLE_2196, ARTICLE_0774, ART..."
1412,USER_1418,"[ARTICLE_2672, ARTICLE_0012, ARTICLE_0906, ART..."
1413,USER_1419,"[ARTICLE_0893, ARTICLE_2775, ARTICLE_1608, ART..."


In [35]:
recommendations_word2vec2 = recommendations_word2vec.explode('articleID').reset_index(drop=True)


In [36]:
recommendations_word2vec2

,userID,articleID
0,USER_0000,ARTICLE_2572
1,USER_0000,ARTICLE_0150
2,USER_0000,ARTICLE_2461
3,USER_0000,ARTICLE_1360
4,USER_0000,ARTICLE_0252
...,...,...
7070,USER_1420,ARTICLE_0767
7071,USER_1420,ARTICLE_2196
7072,USER_1420,ARTICLE_0774
7073,USER_1420,ARTICLE_1323


In [ ]:
recommendations_word2vec2.to_csv(data_path + 'submission_w2v_v3.csv')